In [ ]:
# 1. Tesseract OCR 엔진 설치 (권한/결제 불필요)
!sudo apt install tesseract-ocr
!pip install pytesseract ics
print("Tesseract OCR 및 파이썬 라이브러리 설치 완료.")

# 기존 Vision API 관련 코드는 삭제해도 무방하지만,
# 만약 Vision API 클라이언트가 초기화되어 있다면 에러가 발생할 수 있습니다.
# 다음 코드에서 client 객체를 사용하지 않도록 수정합니다.

In [ ]:
import io
from google.cloud import vision
from google.colab import files

# Vision API 클라이언트 초기화
client = vision.ImageAnnotatorClient()

def extract_text_from_upload():
    """사용자에게 이미지 파일을 업로드하도록 요청하고 OCR 텍스트를 반환합니다."""

    # Colab의 files.upload() 기능으로 파일을 업로드
    uploaded = files.upload()

    if not uploaded:
        print("파일이 업로드되지 않았습니다.")
        return None

    # 업로드된 파일 이름을 가져옴
    file_name = list(uploaded.keys())[0]
    content = uploaded[file_name]

    # Vision API 호출을 위한 Image 객체 생성
    image = vision.Image(content=content)

    # DOCUMENT_TEXT_DETECTION을 사용하여 정확한 텍스트를 추출
    response = client.text_detection(image=image)
    texts = response.text_annotations

    if texts:
        # 첫 번째 텍스트 어노테이션이 전체 텍스트입니다.
        full_text = texts[0].description
        print("\n--- OCR 추출된 전체 텍스트 ---")
        print(full_text)
        print("-----------------------------\n")
        return full_text
    else:
        print("이미지에서 텍스트를 추출하지 못했습니다.")
        return ""

# (테스트용)
# ocr_result = extract_text_from_upload()

In [ ]:
import re
from datetime import datetime, timedelta
from ics import Calendar, Event

def parse_schedule_python(text):
    """OCR 텍스트에서 일정 정보를 추출합니다."""
    today = datetime.now()
    start_date = None
    title = '캡처된 일정'

    # Python 정규식 패턴 (한국어 날짜/시간)
    # 예: 2025.12.10 (수) 오후 3시 30분
    date_regex = re.compile(
        r'(\d{1,4}[년\.]\s?\d{1,2}[월\.]\s?\d{1,2}일?|\d{1,2}[월\.]\s?\d{1,2}일?)\s*([가-힣]{1,2}요일)?\s*(오전|오후)?\s*(\d{1,2})\s*시\s*(\d{1,2})?\s*분?',
        re.IGNORECASE
    )

    match = date_regex.search(text)

    if match:
        date_part = match.group(1) # '2025년 12월 10일' 또는 '12월 10일'
        ampm = match.group(3)      # '오전' 또는 '오후'
        hour_str = match.group(4)  # '3'
        minute_str = match.group(5) # '30' 또는 None

        # 1. 날짜 추출 (년, 월, 일)
        date_nums = re.findall(r'\d+', date_part)

        year = today.year
        month = today.month
        day = today.day

        if len(date_nums) == 3:
            year, month, day = map(int, date_nums)
        elif len(date_nums) == 2:
            month, day = map(int, date_nums)
            # 월이 현재 월보다 작다면, 내년으로 가정
            if month < today.month:
                year += 1

        # 2. 시간 추출 및 보정
        hour = int(hour_str)
        minute = int(minute_str) if minute_str else 0

        if ampm and ampm.lower() == '오후' and hour < 12:
            hour += 12
        elif ampm and ampm.lower() == '오전' and hour == 12: # 오전 12시는 0시
            hour = 0

        try:
            # 최종 datetime 객체 생성
            start_date = datetime(year, month, day, hour, minute)
        except ValueError as e:
            print(f"날짜 생성 오류: {e}")
            return None

    # 3. 제목 추출 (첫 번째 의미있는 줄)
    lines = [line.strip() for line in text.split('\n') if line.strip()]
    if lines:
        first_significant_line = next((line for line in lines if 5 < len(line) < 50), None)
        if first_significant_line:
            title = first_significant_line

    # 4. 결과 반환
    return {
        'title': title,
        'start_date': start_date,
        'end_date': start_date + timedelta(hours=1) if start_date else None, # 1시간 기본값
        'description': text # 전체 텍스트를 설명에 포함
    }

def generate_ics_file(event_data, output_filename="schedule_export.ics"):
    """파싱된 데이터를 사용하여 .ics 파일을 생성하고 다운로드 링크를 제공합니다."""
    if not event_data or not event_data['start_date']:
        print("유효한 일정 정보가 없습니다.")
        return

    c = Calendar()
    e = Event()
    e.name = event_data['title']
    e.begin = event_data['start_date']
    e.end = event_data['end_date']
    e.description = event_data['description']

    c.events.add(e)

    # ics 파일 저장
    with open(output_filename, 'w') as f:
        f.writelines(c)

    print(f"\n✅ ICS 파일 생성 성공: '{output_filename}'")

    # Colab에서 파일을 사용자 컴퓨터로 다운로드
    files.download(output_filename)

# (테스트용)
# parsed_data = parse_schedule_python(ocr_result)
# if parsed_data and parsed_data['start_date']:
#     generate_ics_file(parsed_data)

In [ ]:
# 기존 3단계에서 정의된 함수들은 그대로 유지합니다.
# (parse_schedule_python, generate_ics_file)

from ics import Calendar, Event
import io
from google.colab import files
from google.colab import auth

# 인증 확인 (필요 시 다시 실행)
# auth.authenticate_user()

def process_multiple_uploads():
    """여러 이미지를 업로드하고, 추출된 일정을 하나의 ICS 파일로 누적하여 저장합니다."""

    # 누적 저장을 위한 Calendar 객체 초기화
    accumulated_calendar = Calendar()
    total_events_count = 0

    print("--- 🔄 일정 누적 처리 시작 ---")

    while True:
        # 사용자에게 파일 업로드를 요청
        print("\n[현재 누적된 일정 수: {}]".format(total_events_count))
        print("다음 일정 이미지 파일을 선택하거나, 완료하려면 '취소'를 눌러주세요.")

        uploaded = files.upload()

        if not uploaded:
            # 사용자가 파일 선택을 취소하거나 더 이상 업로드하지 않을 때 종료
            if total_events_count > 0:
                print("\n**업로드 완료. 누적된 일정 {}개로 ICS 파일을 생성합니다.**".format(total_events_count))
                break
            else:
                print("\n❌ 업로드된 이미지가 없습니다. 종료합니다.")
                return

        file_name = list(uploaded.keys())[0]
        content = uploaded[file_name]

        # 1. OCR 텍스트 추출
        try:
            image = vision.Image(content=content)
            response = client.text_detection(image=image)
            ocr_result = response.text_annotations[0].description if response.text_annotations else ""

            if not ocr_result:
                 print(f"⚠️ 경고: '{file_name}'에서 텍스트를 추출하지 못했습니다. 건너뜁니다.")
                 continue

            print(f"\n[✅ '{file_name}' 텍스트 추출 완료]")
            # print(ocr_result) # 디버깅용

            # 2. 텍스트 파싱
            parsed_data = parse_schedule_python(ocr_result)

            if parsed_data and parsed_data['start_date']:
                # 3. 누적 Calendar 객체에 Event 추가
                e = Event()
                e.name = parsed_data['title']
                e.begin = parsed_data['start_date']
                e.end = parsed_data['end_date']
                e.description = parsed_data['description']

                accumulated_calendar.events.add(e)
                total_events_count += 1

                print(f"   -> 추출 성공: 제목 '{e.name}', 시간 {e.begin.datetime.strftime('%Y-%m-%d %H:%M')}")
            else:
                print(f"   -> ❌ 파싱 실패: '{file_name}'에서 유효한 날짜/시간 정보를 찾을 수 없습니다.")

        except Exception as e:
            print(f"   -> ❌ 처리 중 예외 발생 ('{file_name}'): {e}")

    # 최종 ICS 파일 생성 및 다운로드
    output_filename = "accumulated_schedule_export.ics"
    if total_events_count > 0:
        with open(output_filename, 'w') as f:
            f.writelines(accumulated_calendar)

        print(f"\n🎉 최종 ICS 파일 생성 성공: '{output_filename}' (총 {total_events_count}개 일정)")
        files.download(output_filename)
    else:
        print("\n최종적으로 누적된 일정이 없으므로 파일을 생성하지 않습니다.")

# 누적 처리 함수 실행
process_multiple_uploads()

--- 🔄 일정 누적 처리 시작 ---

[현재 누적된 일정 수: 0]
다음 일정 이미지 파일을 선택하거나, 완료하려면 '취소'를 눌러주세요.


Saving 화면 캡처 2025-12-10 152319.png to 화면 캡처 2025-12-10 152319 (2).png
   -> ❌ 처리 중 예외 발생 ('화면 캡처 2025-12-10 152319 (2).png'): 403 Cloud Vision API has not been used in project 522309567947 before or it is disabled. Enable it by visiting https://console.developers.google.com/apis/api/vision.googleapis.com/overview?project=522309567947 then retry. If you enabled this API recently, wait a few minutes for the action to propagate to our systems and retry. [reason: "SERVICE_DISABLED"
domain: "googleapis.com"
metadata {
  key: "service"
  value: "vision.googleapis.com"
}
metadata {
  key: "serviceTitle"
  value: "Cloud Vision API"
}
metadata {
  key: "containerInfo"
  value: "522309567947"
}
metadata {
  key: "consumer"
  value: "projects/522309567947"
}
metadata {
  key: "activationUrl"
  value: "https://console.developers.google.com/apis/api/vision.googleapis.com/overview?project=522309567947"
}
, locale: "en-US"
message: "Cloud Vision API has not been used in project 522309567947 before or